In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
# Load the datasets
train_data = pd.read_csv('C://Users//Hp//Documents//WIDs//train.csv')
test_data = pd.read_csv('C://Users//Hp//Documents//WIDs//test.csv')

In [3]:
# Data Exploration
print(train_data.head())
print(train_data.info())
print(train_data.describe())
print(train_data['metastatic_diagnosis_period'].value_counts())

   patient_id patient_race  payer_type patient_state  patient_zip3   Region  \
0      268700          NaN  COMMERCIAL            AR           724    South   
1      484983        White         NaN            IL           629  Midwest   
2      277055          NaN  COMMERCIAL            CA           925     West   
3      320055     Hispanic    MEDICAID            CA           900     West   
4      190386          NaN  COMMERCIAL            CA           934     West   

             Division  patient_age patient_gender    bmi  ...  \
0  West South Central           39              F    NaN  ...   
1  East North Central           55              F  35.36  ...   
2             Pacific           59              F    NaN  ...   
3             Pacific           59              F    NaN  ...   
4             Pacific           71              F    NaN  ...   

  Average of Apr-18 Average of May-18 Average of Jun-18 Average of Jul-18  \
0             52.55             74.77             79.96  

In [4]:
# Data Preprocessing
# Handling missing values
missing_values = train_data.isnull().sum()
print(missing_values)


patient_id                        0
patient_race                   6657
payer_type                     1765
patient_state                     0
patient_zip3                      0
                               ... 
Average of Sep-18                 7
Average of Oct-18                 7
Average of Nov-18                12
Average of Dec-18                33
metastatic_diagnosis_period       0
Length: 152, dtype: int64


In [5]:
# Check for unique values in 'metastatic_diagnosis_period' column
print(train_data['metastatic_diagnosis_period'].value_counts())


metastatic_diagnosis_period
0      3126
28      140
14      140
21      134
7       129
       ... 
341       6
345       6
333       5
334       5
137       5
Name: count, Length: 366, dtype: int64


In [6]:
# Handle KeyError by checking column names
print(train_data.columns)

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'Region', 'Division', 'patient_age', 'patient_gender',
       'bmi',
       ...
       'Average of Apr-18', 'Average of May-18', 'Average of Jun-18',
       'Average of Jul-18', 'Average of Aug-18', 'Average of Sep-18',
       'Average of Oct-18', 'Average of Nov-18', 'Average of Dec-18',
       'metastatic_diagnosis_period'],
      dtype='object', length=152)


In [7]:
# Data Preprocessing
# Check for column names in train_data and test_data
print("Columns in train_data:", train_data.columns)
print("Columns in test_data:", test_data.columns)


Columns in train_data: Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'Region', 'Division', 'patient_age', 'patient_gender',
       'bmi',
       ...
       'Average of Apr-18', 'Average of May-18', 'Average of Jun-18',
       'Average of Jul-18', 'Average of Aug-18', 'Average of Sep-18',
       'Average of Oct-18', 'Average of Nov-18', 'Average of Dec-18',
       'metastatic_diagnosis_period'],
      dtype='object', length=152)
Columns in test_data: Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'Region', 'Division', 'patient_age', 'patient_gender',
       'bmi',
       ...
       'Average of Mar-18', 'Average of Apr-18', 'Average of May-18',
       'Average of Jun-18', 'Average of Jul-18', 'Average of Aug-18',
       'Average of Sep-18', 'Average of Oct-18', 'Average of Nov-18',
       'Average of Dec-18'],
      dtype='object', length=151)


In [8]:
import sweetviz as sv

my_report = sv.analyze(train_data)
my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

C:\Users\Hp\PycharmProjects\testpythonProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[Step 3/3] Generating associations graph     |                                                                                                             | [  0%]   00:00 -> (? left)C:\Users\Hp\PycharmProjects\testpythonProject\venv\Lib\site-packages\sweetviz\graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_dataframe[feature] = pd.Series(dtype=float)
C:\Users\Hp\PycharmProjects\testpythonProject\venv\Lib\site-packages\sweetviz\graph_associations.py:219: Perfor

Done! Use 'show' commands to display/save.   |█████████████████████████████████████████████████████████████████████████████████████████████████████████| [100%]   00:54 -> (00:00 left)


Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
import pandas as pd
from ydata_profiling import ProfileReport

profile = ProfileReport(train_data, title="Profiling Report")
profile.to_file("report.html")

C:\Users\Hp\PycharmProjects\testpythonProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Summarize dataset:  97%|█████████████████████████████████████████████████████████████████████████████████████████████▉   | 154/159 [00:20<00:00, 48.38it/s, Calculate auto correlation]C:\Users\Hp\PycharmProjects\testpythonProject\venv\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'West South Central'')
  warnings.warn(


Export report to file: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.70s/it]
